In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:
#To stop the Thymio
%%run_python
motor_right=0
motor_left=0

UsageError: Line magic function `%%run_python` not found.


In [2]:
%%run_python

speed0 = 100     # nominal speed
speedGain = 2     # gain used with ground gradient
obstSpeedGain = [6, 4, -2, -6, -8]    # /100
                  # gains used with front proximity sensors 0..4

timer_period[0] = 10  # 10ms sampling time


def timer0():
    global prox_ground_delta, prox_horizontal, speed0, speedGain,obstSpeedGain, motor_left_target, motor_right_target
    
    # speed based on nominal velocity and ground (gradient due to goal)
    spLeft = speed0 
    spRight = speed0 
    
    # adjustment for obstacles ("gradient" due to obstacles)
    for i in range(5):
        spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
        spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100
    # motor control
    motor_left_target = spLeft
    motor_right_target = spRight

In [ ]:
async def rotate_robot(client):
    global x1 ,y1 ,x2 ,y2, motor_left_target, motor_right_target,angle,move_list,j
    angle = calculate_angle_degrees(x1, y1, x2, y2)
    print('j =', j)
    
    if move_list[j] == angle :
        angle = 0
    else :
        move_list[j+1] = angle
        angle = angle - move_list[j]
        j = j+1
    print(move_list)
    
    if angle < 0 :
        angle = -angle
        time_rotation = (4.5 * angle) / 180
        v = {
            "motor.left.target": [int(100)],
            "motor.right.target": [int(-100)],
    }
        await node.set_variables(v)
        await asyncio.sleep(time_rotation)
    else:
        time_rotation = (4.5 * angle) / 180
        if time_rotation == 0 :
            v_reverse = {
            "motor.left.target": [int(100)],
            "motor.right.target": [int(100)],
        }
        else :
            v_reverse = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v_reverse)
        await asyncio.sleep(time_rotation)
    v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
    await node.set_variables(v_stop)

In [14]:
%%run_python
import asyncio
speed = 100
speed0 = 100     # nominal speed
speedGain = 2      # gain used with ground gradient
obstThrL = 10      # low obstacle threshold to switch state 1->0
obstThrH = 20      # high obstacle threshold to switch state 0->1
obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)

state = 1         # 0=gradient, 1=obstacle avoidance
obst = [0,0,0,0,0]       # measurements from left and right prox sensors

timer_period[0] = 10   # 10ms sampling time
turn_direction = 0

@onevent 
def timer0():
    global prox_ground_delta, prox_horizontal, motor_left_target, motor_right_target, state, obst, obstThrH, obstThrL, obstSpeedGain, speed0, speedGain,turn_direction
    # acquisition from ground sensor for going toward the goal
    diffDelta = prox_ground_delta[1] - prox_ground_delta[0]

    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[3], prox_horizontal[4], prox_horizontal[2]]
    # tdmclient does not support yet multiple and/or in if statements:
    if state == 0: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (obst[0] > obstThrH):
            state = 1
        elif (obst[1] > obstThrH):
            state = 1
        elif (obst[2] > obstThrH):
            state = 1
        elif (obst[3] > obstThrH):
            state = 1
        elif (obst[4] > obstThrH):
            state = 1
    elif state == 1:
        if obst[0] < obstThrL:
            if obst[1] < obstThrL :
                if obst[2] < obstThrL :
                    if obst[3] < obstThrL :
                        if obst[4] < obstThrL :
                # switch from obst avoidance to goal tracking if obstacle got unseen
                            if turn_direction == 1 :
                                motor_left_target = speed0 
                                motor_right_target = -speed0 
                                state = 0
                                elif turn_direction == -1 :
                                    motor_left_target = -speed0 
                                    motor_right_target = speed0
                                    state = 0
                                    else
                                    state = 0
                                
    
    if  state == 0 :
        # goal tracking: turn toward the goal
        leds_top = [0,0,0]
        motor_left_target = speed0 
        motor_right_target = speed0
        speed = (motor_left_target+motor_right_target//2)//3
        print(speed)
    else:
        leds_top = [30,30,30]
        # obstacle avoidance: accelerate wheel near obstacle
    if (obst[0] > obstThrH and obst[1] > obstThrH) :
        motor_left_target = speed0 + obstSpeedGain * (obst[0] // 75)
        motor_right_target = speed0 + obstSpeedGain * (obst[3] // 75)
        turn_direction = -1
    elif (obst[2] > obstThrH and obst[3] > obstThrH) :
        motor_left_target = speed0 + obstSpeedGain * (obst[1] // 75)
        motor_right_target = speed0 + obstSpeedGain * (obst[3] // 75)
        turn_direction = 1
    elif (obst[4] > obstThrH and obst[1] > obstThrH) :
        motor_left_target = speed0 + obstSpeedGain * (obst[1] // 75)
        motor_right_target = speed0 + obstSpeedGain * (obst[3] //75)
        turn_direction = -1
    elif (obst[4] > obstThrH and obst[2] > obstThrH) :
        motor_left_target = speed0 + obstSpeedGain * (obst[0] // 75)
        motor_right_target = speed0 + obstSpeedGain * (obst[2] // 75)
        turn_direction = 1
    
        print("Current speed:", speed, "cm/s")



    
        

        

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    raise error
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 59, in read_packet
    packet_len = self.read_uint32()
                 ^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 46, in read_uint32
    b = self.io.read(4)
        ^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 99, in read
  